In [ ]:
# New ones
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words
import gensim
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.models.nmf import Nmf
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel
from operator import itemgetter # Already part of base Python
from sentence_transformers import SentenceTransformer

In [ ]:
# Load the full twitter corpus
key_tweets = pd.read_csv('combined_key.csv', index_col = 0)

In [ ]:
documents = key_tweets['text_cleaned_string'].values

In [ ]:
# Read in the sentence transformer model and encode the text with it
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
encoded_data = model.encode(documents)

In [ ]:
# Create the k-means model and fit it to our text
km = KMeans(n_clusters = 5, random_state = 42) # Using 5 clusters here
km.fit(encoded_data)

In [ ]:
# Define a function to return topic per document 
def doc_topic_id(all_training, km, num_clusters):
    clusters = km.labels_.tolist()
    docs = {'document_text':all_training, 'topic_id':clusters}
    frame = pd.DataFrame(docs, index = [clusters])
    for cluster in range(0, num_clusters):
        this_cluster_text = \
        frame[frame['topic_id'] == cluster]
        all_text = \
        " ".join(this_cluster_text['document_text'].astype(str))
    return frame

In [ ]:
# Assign to a dataframe and add original index values from the emails dataframe
key_tweets['doc_id'] = key_tweets.index # Pull out index values into column

topics_bert = doc_cluster_id(documents, km, 5) # Create dataframe
topics_bert = topics_bert.set_index(key_tweets['doc_id']) # Assign doc_id from emails
topics_bert

In [ ]:
# Group by topic and get most frequent 10 words per topic
topics_bert_grouped = topics_bert.groupby(['cluster_id'])['document_text'].apply(','.join).reset_index()
topics_bert_grouped['tokens'] = topics_bert_grouped['document_text'].apply(re_tokenizer.tokenize)

topics_bert_words = []

for i in topics_bert_grouped['tokens']:
    counter = Counter(i)
    topics_bert_words.append(counter.most_common(10))

topics_bert_words = pd.DataFrame(topics_bert_words)
topics_bert_words = topics_bert_words.transpose()
topics_bert_words.columns = ['topic_1', 'topic_2', 'topic_3', 'topic_4', 'topic_5']
topics_bert_words